In [ ]:
import numpy as np
import pandas as pd
import time 
import datetime as datetime
import glob
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
np.random.seed(1)

In [ ]:
Pad = pd.read_csv('PadData_v2.csv')
Pad.head()

In [ ]:
T = 3617
m = 10
to_draw = np.sort(Pad['timestamp'].unique())
ccy = np.sort(Pad['currency pair'].unique())
min_history = 1000 # min episode length
    
def generate_episode(n,cur):
    _max = to_draw.shape[0]
    _end = min(n+T, _max)
    timeframe = to_draw[n:_end]
    other_bid = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    other_ask = np.zeros((timeframe.shape[0],ccy.shape[0]-1))
    i = 0
    for elem in ccy:
        tmp = Pad[Pad['currency pair'] == elem]
        if elem == cur:
            target_bid = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            target_ask = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
        else:
            other_bid[:,i] = tmp[tmp.timestamp.isin(timeframe)]['bid price'].values
            other_ask[:,i] = tmp[tmp.timestamp.isin(timeframe)]['ask price'].values
            i += 1
    target_mid = np.mean(np.array([target_bid, target_ask]),axis=0)
    other_mid = np.mean(np.array([other_bid, other_ask]),axis=0)
    return target_bid, target_ask, other_mid, target_mid

def features(price_path,m):
    features = np.zeros((price_path.shape[0]-m,m))
    for i in range(m):
        features[:,i] = (np.log(price_path) - np.log(np.roll(price_path, i+1)))[m:]
    return features

def BollingerBands(price_path,m):
    _mean = pd.Series(price_path).rolling(window=m).mean().values[m:]
    _std = pd.Series(price_path).rolling(window=m).std().values[m:]
    upper = _mean + 2*_std
    lower = _mean - 2*_std
    return _mean, upper, lower

def get_features(target_bid, target_ask, other_mid, target_mid, m):
    feature_span = features(target_bid,m)
    feature_span = np.append(feature_span, features(target_ask,m), axis = 1)
    for i in range(other_mid.shape[1]):
        feature_span = np.append(feature_span, features(other_mid[:,i],m), axis = 1)
    _mean, upper, lower = BollingerBands(target_mid,m)
    tech_features = np.array([_mean, upper, lower]).T
    return feature_span, tech_features

def draw_episode(m, cur, min_history):
    '''
    Input:
        m, number of lag returns z_1,...z_m
        cur, currency pair that we target to trade
        min_history, min length of a valid episode
    '''
    n = np.random.randint(to_draw.shape[0] - min_history)
    target_bid, target_ask, other_mid, target_mid = generate_episode(n,cur)
    feature_span, tech_features = get_features(target_bid, target_ask, other_mid, target_mid, m)
    return target_bid, target_ask, feature_span, tech_features